#Osservazioni codice#



*   **check_time**: devo usare anche il domain del sito per forza?
*   **fetch**: devo usare un if se è la prima volta che visito l'url cosi richiamo il robots.txt? quando devo controllare la sitema etc.?
*  **classe** crea una classe useragent così è più facile gestire tutte le funzioni!!!

#Librerie#

In [ ]:
#url di prova
# ha sitemap-index ma non priority etc.
start_url_uno = "https://www.dragopublisher.com/it/"
# non ha sitemap
start_url_due = "https://www.wildraccoon.it/shop/"
# sito giocattolo ha tutto
start_url = "https://nonciclopedia.org/wiki/Wikipedia"

In [2]:
import re #per gestire le regex --- non credo servirà
import pandas as pd
import urllib.robotparser #per gestire il file robot.txt
import xml.etree.ElementTree as ET  #per gestire i file xml
import requests #per http
import time #per gestire il tempo
from bs4 import BeautifulSoup #per gestire il parsing del html (si può usare anche per xml)

In [3]:
def chech_roobots(url,useragent="*"):
  '''
  input: useragent (default: *, generico), url sito da analizzare
  output: check,path_disallow,crawl_delay,request_rate,sitemap

  controlla il file robot.txt associato all'url
  '''

  rfl = urllib.robotparser.RobotFileParser()
  # aggiungo all'url selezionato l'estensione corretta per leggere il file robots.txt
  new_url = url + "/robots.txt"

  rfl.set_url(new_url)
  #legge il file robots.txt e controlla se lo useragent puo effettuare il fetch dei dati
  rfl.read()
  check = rfl.can_fetch(useragent,new_url)
  if check:
    print("{} il sito puo effettuare il fetch dei dati".format(check))
  else:
    print("il sito non puo effettuare il fetch dei dati")

  # salva le path non accessibili (Disallow:...)
  path_disallow = rfl.parse("Disallow")
  print("path non accessibili: {}".format(path_disallow))

  # salva il craw_delay: intervallo temporale tra richieste dello stesso useragent
  crawl_delay = rfl.crawl_delay(useragent)
  print("crawl delay: {}".format(crawl_delay))

  # salva il request_rate: numero di richieste ogni tot secondi (tuple (r,s))
  request_rate = request_rate(useragent)
  print("request rate: {}".format(request_rate))

  # salva la sitemap
  sitemap = rfl.site_maps()
  print("sitemap: {}".format(sitemap))



  return check,path_disallow,crawl_delay,request_rate,sitemap

#Fetcher#
gestione richieste e download pagine web

In [ ]:
class UserAgentPolicy
  '''
    classe per identificare gli useragent e salvare di volta in volta le informazioni necessarie per gestire le richieste
  '''
  def __init__(self, name, base_url, path_dissalow, crawl_delay, request_rate, n_request,last_access, header):
        self.base_url = base_url
        self.name = name
        self.path_dissalow = path_dissalow
        self.crawl_delay = crawl_delay
        self.request_rate = request_rate
        self.n_request = n_request
        self.last_access = last_access
        self.header = header

In [ ]:
#Useragents di prova#

devo sistemare il codice pensando a come usare la classe user agent ovunque... spreco di memoria? ---> check necessario

In [4]:
def check_time(crawl_delay=None,last_access=0,request_rate=None, n_requests= 0):
  '''
    controlla che lo useragent rispetti il tempo di craw-delay e il request_rate
  '''

  # check che il bot on faccia più di n richieste in m secondi (= rispetti il request rate)
  # check che il bot aspetti sempre il tempo di crawl delay tra le richieste
  # orologio assoluto non sensibile alle variazioni del system clock, precisione ns

  if request_rate == None && crawl_delay == None:
     delay = 1.5  #valore a caso

  # il crawl delay può essere inferiore all'intervallo id tempo tra le richieste del request_delay  e questo fa si che la frequenza di richieste nonvenga rispettata
  if request_rate[0]/request_rate[1] > crawl_delay:
    delay = request_rate[0]/request_rate[1]
  else:
    delay = crawl_delay

  now = time.monotonic_ns()
  wait = max(0, (last_access + delay) - now)
  if wait > 0:
      time.sleep(wait)
  last_access = time.monotonic_ns()
  return

In [ ]:
def fetch(url,UserAgent='*',craw_delay=5, last_access=0, request_rate=(3,10)): #hp buttato valori di default a caso per ora
  '''
  '''
  # per fare test e simulare il comportamento di più useragent (non so se metterli nelle classi)
  useragent_dict = {'Googlebot':'Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)',
                    'Bingbot':'Mozilla/5.0 (compatible; Bingbot/2.0; +http://www.bing.com/bingbot.htm)',
                    'Slurp':'Mozilla/5.0 (compatible; Yahoo! Slurp; http://help.yahoo.com/help/us/ysearch/slurp)',
                    'DuckDuckbot':'DuckDuckBot/1.0; (+http://duckduckgo.com/duckduckbot.html)',
                    'Yandex':'Mozilla/5.0 (compatible; YandexBot/3.0; +http://yandex.com/bots)',
                    '*': 'Mozilla/5.0 (compatible; MyBot/1.0; +http://example.com/bot)'} #---> non so se devo mettere solo '*' anche su questo header (generato con ChatGPT)


  # controllo che lo user non esegua troppe richieste allo stesso DNS
  check_time(crawl_delay=craw_delay,last_access=last_access)

  # costuisco l'header specifico per lo useragent richiesto(usato per test, non semrpe necessario)
  headers = {
        'User-Agent': useragent_dict[useragent]
    }
  # contatto il sito
  response = requests.get(url, headers=headers, timeout=10)
  if response.status_code != 200:
     print("Errore nel recupero:", url)
  return

In [ ]:
def is_allowed(url,path_disallow,useragent='*'):


#Main fetcher esempio#

# Parser#
analisi contenuto della pagina, estrazione testo e link e normalizzazione link trovati

In [ ]:
def parse_sitemap(sitemap_list):
    '''
    Prende in input una lista di sitemap URLs e restituisce un DataFrame
    con: URL, priorità e frequenza di aggiornamento.
    '''
    sitemap_all = pd.DataFrame(columns=['url','priority','update'])

    # Standard XML Namespace ufficiale del protocollo Sitemap, serve per identificare il vocabolario e la struttura usata nel file xml
    namespace = {'ns': 'http://www.sitemaps.org/schemas/sitemap/0.9'}

    def parse_single_sitemap(sitemap_url):

        response = requests.get(sitemap_url)
        if response.status_code != 200:
           print("Errore nel recupero:", sitemap_url)
           return []

        # Controllo che la sitemap risponda (da aggiungere la possibilità di decompressione se necessaria)
        root = ET.fromstring(response.content)

        # Se è una sitemap index (contiene altre sitemap)
        if root.tag.endswith('index'):
            sitemap_urls = [loc.text for loc in root.findall('ns:sitemap/ns:loc', namespace)]
            all_entries = []
            for sub_url in sitemap_urls:
                all_entries.extend(parse_single_sitemap(sub_url))
            return all_entries

        # Se è una sitemap normale (contiene solo URL)
        elif root.tag.endswith('urlset'):
            entries = []
            for url in root.findall('ns:url', namespace):
                loc = url.find('ns:loc', namespace)
                priority = url.find('ns:priority', namespace)
                changefreq = url.find('ns:changefreq', namespace)

                entries.append({
                    'url': loc.text if loc is not None else '',
                    'priority': priority.text if priority is not None else None,
                    'update': changefreq.text if changefreq is not None else None
                })
            return entries
        else:
            print(f"Tipo di sitemap non riconosciuto: {root.tag}")
            return []

    # Analizza ogni sitemap passata
    all_data = []

    if sitemap_list is None:
        print("Nessuna sitemap presente")
        return sitemap_all

    for sitemap_url in sitemap_list:
        if sitemap_url is None:
            print("Nessuna sitemap presente")
            continue
        all_data.extend(parse_single_sitemap(sitemap_url))

    # Crea DataFrame finale
    sitemap_all = pd.DataFrame(all_data)
    return sitemap_all


In [ ]:
def parse_page(url,sitemaps_urls,path_disallow):
  '''
    dato un url di input cerco le pagine non indicizzate accessibili:
    - se sono in una directory privata li scarto
    - se gia presenti nella sitemap, sono già inidicizzate e non salvo i link (duplicati)
    - se non presenti salvo gli url permessi dal file robots.txt
  '''
  #simulo uno user-agent
  headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0 Safari/537.36'
    }

  #contatto la pagina
  response = requests.get(url, headers=headers, timeout=10)
  if response.status_code != 200:
     print("Errore nel recupero:", url)
     print(response)

  #parsing pagina html
  soup = BeautifulSoup(response.text, 'html.parser')

  urls = []
  # Elementi con attributo href
  for tag in soup.find_all(href=True):
      urls.append(tag['href'])

  # Elementi con attributo src
  for tag in soup.find_all(src=True):
      urls.append(tag['src'])

  # controlla se i nuovi url sono veramente accessibili per il web scraping (no in disallow)
  if path_disallow != None:
    for url in urls:
      for path in path_disallow:
        if path in url:
         urls.remove(url)

  #controlla se ci sono duplicati nella sitemap se c'è
  if sitemaps_urls != None:
    def_urls = sitemaps_urls
    for url in urls:
      if url not in def_urls:
         def_urls.append(url)
  else:
    def_urls = urls

  return def_urls



In [ ]:
def parse_link():
  return

In [ ]:
def parse_text():
  return

#Main prove#

In [ ]:
class BotPolicy:
    def __init__(self, domain, user_agent):
        self.domain = domain
        self.user_agent = user_agent
        self.parser = self._load_robots()
        self.last_request_time = None
        self.request_counter = 0
        self.rate_window_start = time.monotonic()

    def _load_robots(self):
        parser = RobotFileParser()
        parser.set_url(f"{self.domain}/robots.txt")
        parser.read()
        return parser

    def can_fetch(self, path):
        return self.parser.can_fetch(self.user_agent, self.domain + path)

    def crawl_delay(self):
        return self.parser.crawl_delay(self.user_agent)

    def request_rate(self):
        return self.parser.request_rate(self.user_agent)

    def wait(self):
        # stessa logica vista prima: crawl_delay e request_rate
        pass

class UserAgentManager:
    def __init__(self):
        self.policies = {}  # chiave: (domain, user_agent)

    def get_policy(self, domain, user_agent):
        key = (domain, user_agent)
        if key not in self.policies:
            self.policies[key] = BotPolicy(domain, user_agent)
        return self.policies[key]


In [ ]:
import time
import requests
from urllib.robotparser import RobotFileParser
from urllib.parse import urlparse

class RespectfulCrawler:
    def __init__(self, base_url, user_agent="MyBot"):
        self.base_url = base_url
        self.user_agent = user_agent
        self.domain = urlparse(base_url).scheme + "://" + urlparse(base_url).netloc
        self.parser = RobotFileParser()
        self.parser.set_url(self.domain + "/robots.txt")
        self.parser.read()
        self.last_request_time = None
        self.request_counter = 0
        self.start_window = time.monotonic()

    def is_allowed(self, path):
        return self.parser.can_fetch(self.user_agent, self.domain + path)

    def wait_respecting_robots(self):
        # Handle crawl-delay
        delay = self.parser.crawl_delay(self.user_agent)
        if delay and self.last_request_time:
            elapsed = time.monotonic() - self.last_request_time
            if elapsed < delay:
                time_to_wait = delay - elapsed
                print(f"[WAIT] Crawl-delay: sleeping for {time_to_wait:.2f} seconds")
                time.sleep(time_to_wait)

        # Handle request-rate
        rate = self.parser.request_rate(self.user_agent)
        if rate:
            now = time.monotonic()
            if now - self.start_window > rate.seconds:
                self.start_window = now
                self.request_counter = 0
            self.request_counter += 1
            if self.request_counter > rate.requests:
                reset_time = self.start_window + rate.seconds - now
                print(f"[WAIT] Request-rate: sleeping for {reset_time:.2f} seconds")
                time.sleep(reset_time)
                self.start_window = time.monotonic()
                self.request_counter = 1

        self.last_request_time = time.monotonic()

    def fetch(self, path):
        if not self.is_allowed(path):
            print(f"[BLOCKED] Access to {path} is disallowed by robots.txt")
            return None

        self.wait_respecting_robots()

        headers = {
            'User-Agent': self.user_agent
        }

        url = self.domain + path
        print(f"[FETCH] Getting {url}")
        response = requests.get(url, headers=headers)
        return response

# 🧪 ESEMPIO USO

if __name__ == "__main__":
    crawler = RespectfulCrawler("https://example.com", user_agent="MyBot")

    # Lista di pagine da visitare
    paths = ["/", "/about", "/contact"]

    for path in paths:
        response = crawler.fetch(path)
        if response and response.status_code == 200:
            print(f"[OK] {path} - {len(response.text)} bytes")
        elif response:
            print(f"[ERROR] {path} - Status: {response.status_code}")
